In [3]:
## Data scrapping prpm
from PyClasses.Scrapers.PRPMScraper import PRPMScraper
from PyClasses.Scrapers.PERIScraper import PeriScraper
from PyClasses.Preprocessing.PRPMCleaner import PRPMCleaner
prpm_cleaner = PRPMCleaner()
PRPMscrap = PRPMScraper()


word = 'demografidalaman'
result = PRPMscrap.findWordMetaData(word)
print(result)

if result and 'meanings' in result[word]:
    raw_meanings = result[word]['meanings']
    cleaned_meanings = prpm_cleaner.clean_all_meanings(raw_meanings)
    result[word]['meanings'] = cleaned_meanings

# Output cleaned result
print("Cleaned Result:", result)

#found smth can be preprocess if not done here, like there will have '\xa' character in the result string

None
Cleaned Result: None


In [1]:
from datetime import datetime, timedelta
from PyClasses.Scrapers.BHarianScraper import BHarianScraper
from pyspark.sql import SparkSession
from selenium import webdriver

def ArticleDataCollection(fromDate,categoryLink):
    spark = SparkSession\
        .builder\
        .appName("DE-prj")\
        .getOrCreate()

    ## crawl links
    bharianScraper = BHarianScraper()
    timeNow = datetime.now()
    crawlledUrls = bharianScraper.scrapArticleLinks(fromDate,categoryLink)
    url_RDD = spark.sparkContext.parallelize(crawlledUrls)
    if crawlledUrls is None:
        print('failed to get article links')
        return 'fail'

#parallelize process to scrap data
    scrappedData = url_RDD.mapPartitions(BHarianScraper.scrapOneArticle).collect()
    for data in scrappedData:
        if data is None:
            scrappedData.remove(data)

#Save Data into HDFS
    df = spark.createDataFrame(scrappedData)
    df.write.format("parquet").mode("overwrite").save("DE-prj/RawData")
    timeEnd = datetime.now() 
    timeTaken = timeEnd - timeNow
    print(f'Processing Start Time: {timeNow}')
    print(f'Processing End Time: {timeEnd}')
    print(f'Total Time Taken: {timeTaken}')

    return 'success'

ArticleDataCollection(datetime(2024,12,16,13,00,00),'/berita/nasional')

24/12/17 02:44:49 WARN Utils: Your hostname, Liang. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/17 02:44:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/17 02:44:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Article teaser found on page 0.
Current no. of link: 20
Article teaser found on page 1.
Current no. of link: 29


Processing Start Time: 2024-12-17 02:44:51.742272
Processing End Time: 2024-12-17 02:45:26.755842
Total Time Taken: 0:00:35.013570


'success'

In [ ]:
# for link in bharianScraper.relLinkToArticle_all:
#     print(link)
spark = SparkSession\
        .builder\
        .appName("ReadRawData")\
        .getOrCreate()
df_read = spark.read.parquet("DE-prj/RawData")
df_read.show()


df_filter = df_read.where(df_read['title'] == '')
df_filter.show()

print(df_read.count())


# Get the first 5 rows
rows = df_read.take(125)

# Print each row
print(rows[15])

In [ ]:
for data in rows:
    print(data)

# PERIBAHASA STARTS HERE YA !

In [1]:
from PyClasses.Scrapers.PERIScraper import PeriScraper
peri_scrap = PeriScraper()

proverbs_data_AM = peri_scrap.scrape_page(peri_scrap.AM_Url)
print("Proverbs from A–M:")
for data in proverbs_data_AM:
    print(data)
# for proverb, meaning in proverbs_data_AM:
#     print(f"Peribahasa: {proverb} || Maksud: {meaning}")

# proverbs_data_NZ = peri_scrap.scrape_page(peri_scrap.NZ_Url)
# print("\nProverbs from N–Z:")

# for proverb, meaning in proverbs_data_NZ:
#     print(f"Peribahasa: {proverb} || Maksud: {meaning}")

# print(proverbs_data_NZ[0])

24/12/16 21:51:31 WARN Utils: Your hostname, LoopZi. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/16 21:51:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/16 21:51:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Processing Table 1...
Processing Table 2...
Processing Table 3...
Processing Table 4...
Processing Table 5...
Processing Table 6...
Processing Table 7...
Processing Table 8...
Processing Table 9...
Processing Table 10...
Processing Table 11...
Processing Table 12...
Processing Table 13...
Processing Table 14...
Proverbs from A–M:
{'peri': 'ada air adalah ikan', 'meaning': 'Ada penempatan adalah penduduk.'}
{'peri': 'ada aku dipandang hadap tiada aku dipandang belakang', 'meaning': 'Kasih sayang hanya waktu berhadapan saja, setelah berjauhan lalu dilupakan.'}
{'peri': 'ada angin ada pokoknya pohonnya', 'meaning': 'Segala sesuatu mestilah ada asal mulanya.'}
{'peri': 'ada bangkai adalah hering', 'meaning': 'Ada perempuan jahat, adalah lelaki jahat yang mengunjunginya.'}
{'peri': 'ada batang mati adalah cendawan tumbuh', 'meaning': 'Di mana juga kita tinggal, akan ada rezeki kita.'}
{'peri': 'ada batang cendawan tumbuh', 'meaning': 'Tiap-tiap negeri ada undang-undang dan adat resamnya mas